In [1]:
from shapleyAnalysis import *
import sys
import os
import pandas as pd
from torch.nn import DataParallel
from GeneLLM_class import *
## IMPORT MODEL AND TOKENIZER ##
from transformers import BertTokenizerFast

## DATA ##
input_data_path = "clean_genes.csv"
gene_loaded_data = pd.read_csv(input_data_path)
sentences = gene_loaded_data["Summary"].tolist()
geneNames = gene_loaded_data["Gene name"].tolist()
gene_to_idx = {gene:idx for idx, gene in enumerate(geneNames)}
tokenizer_max_length= 512
device = 'cuda:1'

model = pickleLoad('solubilityGO/best_model.pkl')
tokenizer = BertTokenizerFast.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')
save_dir = '/home/dandreas/GeneLLM2/solubilityGO/'
dataset_name = 'clean_genes_solubilityGO'

shap_values = pickleLoad(save_dir + dataset_name + '.pkl')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dandreas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


loading data from solubilityGO/best_model.pkl
loading data from /home/dandreas/GeneLLM2/solubilityGO/clean_genes_solubilityGO.pkl


In [14]:
tokens = tokenizer.encode_plus(sentences[0], max_length = tokenizer_max_length,
                                    padding="max_length",
                                    truncation=True)
print(len(tokens.input_ids),tokens.input_ids)
masked = tokens.input_ids.copy()
for ni,i in enumerate(tokens.input_ids):
    if i>4: masked[ni] = 4
tokens['input_ids'] = masked
print(len(tokens.input_ids),tokens.input_ids)
tokens

512 [2, 2052, 2359, 9937, 1920, 2616, 3689, 18471, 1927, 43, 27756, 13970, 23987, 2213, 1956, 13231, 13693, 18, 1920, 2359, 4863, 2258, 6941, 19302, 2213, 4016, 2455, 1930, 1988, 2455, 1977, 3414, 1958, 6291, 1927, 3689, 7002, 18, 2496, 9220, 5185, 2258, 4624, 2176, 1942, 43, 2487, 7849, 4817, 2899, 1922, 2132, 1956, 3939, 2720, 12592, 8316, 8111, 2308, 2176, 1977, 2222, 3286, 1922, 2791, 21941, 1966, 2486, 1966, 2905, 2991, 1930, 6137, 3148, 3197, 18, 5012, 8490, 2274, 1922, 3137, 4936, 5797, 2413, 7963, 18, 6941, 22163, 4016, 1988, 8016, 5863, 1927, 2024, 3154, 4086, 1930, 5705, 43, 2770, 1922, 1920, 3640, 1927, 1920, 4718, 11612, 16, 10544, 5957, 16, 2024, 8576, 1930, 2024, 13405, 18, 5705, 43, 2770, 1922, 6437, 1915, 1009, 12, 2149, 5173, 11837, 22206, 3148, 13, 3434, 3197, 1922, 9731, 2094, 18, 8016, 5863, 1927, 1920, 4128, 6437, 1915, 1009, 3148, 1930, 1920, 9731, 19, 4402, 2024, 2905, 2991, 3148, 4712, 18, 5705, 43, 2770, 1922, 1920, 3640, 1927, 9731, 2024, 23712, 18, 5705, 43, 

{'input_ids': [2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [6]:
sums0 = []
sums1 = []

modelOutputs0 = []
modelOutputs1 = []
shapSums0 = []
shapSums1 = []

x = np.random.randint(0,1e8)
np.random.seed(x)
print('seed',x)
for i in range(10):    
    s = np.random.randint(0,len(sentences))

    class0 = list(map(lambda x: x[0],shap_values[s].values))
    class1 = list(map(lambda x: x[1],shap_values[s].values))

    shapBaseline0 = shap_values[s].base_values[0]
    shapBaseline1 = shap_values[s].base_values[1]

    model_b0, model_b1 = getClassificationLogOdds(sentences[s], model, tokenizer, tokenizer_max_length, device, mask_all = True)    

    shapSum0 = sum(class0)
    shapSum1 = sum(class1)
    shapSums0.append(shapSum0)
    shapSums1.append(shapSum1)

    modelOutput0, modelOutput1 = getClassificationLogOdds(sentences[s], model, tokenizer, tokenizer_max_length, device)
    modelOutputs0.append(modelOutput0)
    modelOutputs1.append(modelOutput1)

    print(modelOutput0, shapSum0, shapSum0 - (modelOutput0 - shapBaseline0))
    print(modelOutput1, shapSum1, shapSum1 - (modelOutput1 - shapBaseline1))


seed 56775821
-1.3351305 0.45898377895355225 0.0
0.96708345 -0.4343196153640744 3.3306690738754696e-16
-2.8143516 -0.9325379133224486 1.1102230246251565e-16
2.4398274 0.9767072200775148 1.1102230246251565e-16
-1.4627217 0.27866792678833 -5.551115123125783e-17
1.348359 -0.011163473129272442 1.9081958235744878e-17
1.5338789 3.288022756576535 -3.1086244689504383e-15
-1.4938551 -2.869350910186772 -4.440892098500626e-15
-2.7077377 -0.8243190050125127 -4.440892098500626e-16
1.8187894 0.34000253677368186 2.220446049250313e-16
-1.1816595 0.5834690332412713 -6.661338147750939e-16
1.3242524 -0.057814002037048076 2.636779683484747e-16
1.909924 3.703950762748717 -1.3322676295501878e-15
-2.8378987 -4.255536198616023 4.440892098500626e-15
-2.3469048 -0.46091341972351035 3.885780586188048e-16
1.645293 0.1775637865066528 -2.7755575615628914e-17
-1.3749522 0.42134547233581554 1.1102230246251565e-16
0.38060912 -1.0285857021808615 8.881784197001252e-16
1.8315564 3.62526631355286 3.552713678800501e-15
-1.

In [4]:
trunc = lambda x: float(f'{x:.3f}')
truncAll = lambda x: list(map(trunc,x))
print('modelOutputs0:',truncAll(modelOutputs0))
print('shapSums0:    ',truncAll(shapSums0))
print('')
print('modelOutputs1:',truncAll(modelOutputs1))
print('shapSums1:    ',truncAll(shapSums1))

modelOutputs0: [-0.797, -1.853, -1.465, -1.871, -1.337, -2.494, 1.188, -0.214, -1.781, -1.709]
shapSums0:     [0.948, -0.102, 0.27, -0.085, 0.501, -0.696, 2.982, 1.738, 0.01, 0.171]

modelOutputs1: [0.738, 1.249, 1.283, 1.661, 1.182, 1.816, -1.18, -0.187, 1.442, 1.982]
shapSums1:     [-0.634, -0.125, -0.079, 0.259, -0.254, 0.406, -2.582, -1.679, 0.031, 0.524]
